# LangChain Tools + Agents + LangGraph (Good / Better / Best)

This notebook follows **current production best practices (2026)**:

- **Good** → Manual tool execution (explicit, debuggable)
- **Better** → **`create_agent`** (modern LangChain agent API, no executor)
- **Best** → Explicit **LangGraph** workflow (`StateGraph`)

Assumes:
- `langchain` (modern v1+)
- `langchain-core`, `langchain-openai`
- `langgraph` + `langgraph-prebuilt`
- `OPENAI_API_KEY` set


In [1]:
import os
from typing import List

from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    ToolMessage,
    BaseMessage,
)

MODEL = os.environ.get("OPENAI_MODEL", "gpt-4o-mini")
print("Using model:", MODEL)

llm = ChatOpenAI(model=MODEL, temperature=0)
SYSTEM = "You are a helpful assistant. Use tools when appropriate."


Using model: gpt-4o-mini


## 1) Pydantic argument schemas

In [2]:
class WeatherSearchArgs(BaseModel):
    airport_code: str = Field(description="Airport code (e.g., SFO)")


class ArtistSearchArgs(BaseModel):
    artist_name: str = Field(description="Artist name")
    n: int = Field(default=3, description="Number of songs")


## 2) Tools (`@tool`, name inferred from function name)

In [3]:
@tool(description="Get demo weather for an airport.", args_schema=WeatherSearchArgs)
def WeatherSearch(airport_code: str) -> str:
    return f"Weather at {airport_code}: sunny, 23°C"


@tool(description="Get demo songs by an artist.", args_schema=ArtistSearchArgs)
def ArtistSearch(artist_name: str, n: int = 3) -> List[str]:
    demo = {
        "taylor swift": ["Anti-Hero", "Blank Space", "Cruel Summer", "Shake It Off"],
        "daft punk": ["One More Time", "Harder, Better, Faster, Stronger", "Get Lucky"],
    }
    return demo.get(artist_name.lower(), ["(no results)"])[:n]


tools = [WeatherSearch, ArtistSearch]
tool_registry = {t.name: t for t in tools}


## Good: Manual tool execution loop

In [4]:
def run_manual(question: str) -> str:
    bound = llm.bind_tools(tools)

    ai_msg = bound.invoke([
        SystemMessage(content=SYSTEM),
        HumanMessage(content=question),
    ])

    tool_messages = []
    for call in ai_msg.tool_calls or []:
        result = tool_registry[call["name"]].invoke(call["args"])
        tool_messages.append(ToolMessage(content=str(result), tool_call_id=call["id"]))

    final = bound.invoke([
        SystemMessage(content=SYSTEM),
        HumanMessage(content=question),
        ai_msg,
        *tool_messages,
    ])
    return final.content


In [5]:
print(run_manual("what is the weather at SFO?"))
print(run_manual("what are three songs by Taylor Swift?"))


The weather at San Francisco International Airport (SFO) is sunny with a temperature of 23°C.
Here are three songs by Taylor Swift:

1. Anti-Hero
2. Blank Space
3. Cruel Summer


## Better: Modern LangChain Agent (`create_agent`)

This is the **recommended agent API**.
- No `AgentExecutor`
- No deprecated helpers
- Tools auto-run internally


In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=SYSTEM,
)

result = agent.invoke({"messages": [{"role": "user", "content": "what is the weather at SFO?"}]})
print(result["messages"][-1].content)


The weather at San Francisco International Airport (SFO) is sunny with a temperature of 23°C.


In [7]:
result = agent.invoke({"messages": [{"role": "user", "content": "what are three songs by Taylor Swift?"}]})
print(result["messages"][-1].content)


Here are three songs by Taylor Swift:

1. Anti-Hero
2. Blank Space
3. Cruel Summer


## Best: Explicit LangGraph workflow

Use this when you need:
- deterministic routing
- validation / guardrails
- retries or approvals
- long-running workflows


In [8]:
# --- Best: Explicit LangGraph workflow (StateGraph) ---
from typing import Annotated, List
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage

# Bind tools to the model used inside the graph
graph_llm = llm.bind_tools(tools)

# 1) State uses the add_messages reducer so message merges preserve tool_call_id/tool ids
class GraphState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# 2) LLM node returns ONLY the new message; reducer appends it
def llm_node(state: GraphState) -> GraphState:
    resp = graph_llm.invoke(state["messages"])
    return {"messages": [resp]}

tool_node = ToolNode(tools)

def route_after_llm(state: GraphState):
    last = state["messages"][-1]
    if last.tool_calls:
        return "tools"
    return END

sg = StateGraph(GraphState)
sg.add_node("llm", llm_node)
sg.add_node("tools", tool_node)

# Modern entry edge syntax
sg.add_edge(START, "llm")
sg.add_conditional_edges("llm", route_after_llm, {"tools": "tools", END: END})
sg.add_edge("tools", "llm")

app = sg.compile()


In [9]:
out = app.invoke({
    "messages": [
        SystemMessage(content=SYSTEM),
        HumanMessage(content="what is the weather at SFO?"),
    ]
})
print(out["messages"][-1].content)


The weather at San Francisco International Airport (SFO) is sunny with a temperature of 23°C.


In [10]:
out = app.invoke({
    "messages": [
        SystemMessage(content=SYSTEM),
        HumanMessage(content="what are three songs by Taylor Swift?"),
    ]
})
print(out["messages"][-1].content)


Here are three songs by Taylor Swift:

1. Anti-Hero
2. Blank Space
3. Cruel Summer
